<a href="https://colab.research.google.com/github/smallrus-web/AZSII_prz6/blob/main/%D0%90%D1%80%D1%85%D0%B8%D0%BF%D0%BE%D0%B2_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_prz6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Группа: ББМО-01-23

Студент: Архипов Дмитрий Евгеньевич



**1.  Загрузка обученной модели и данных MNIST**




In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model1.fit(train_images, train_labels, epochs=5)

# Сохранение модели
model1.save('mnist_model1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
# Добавление дополнительной размерности для каналов
train_images_cnn = train_images.reshape(-1, 28, 28, 1)
test_images_cnn = test_images.reshape(-1, 28, 28, 1)

model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model2.fit(train_images_cnn, train_labels, epochs=5)

# Сохранение модели
model2.save('mnist_model2.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8789 - loss: 0.4350
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9634 - loss: 0.1255
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9765 - loss: 0.0792
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9826 - loss: 0.0592
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9862 - loss: 0.0439


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9132 - loss: 0.2969
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - accuracy: 0.9830 - loss: 0.0548
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 24ms/step - accuracy: 0.9906 - loss: 0.0315
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9934 - loss: 0.0217
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - accuracy: 0.9961 - loss: 0.0138


**2. Реализация атаки FGSM на первую модель**


In [11]:
import numpy as np
import tensorflow as tf

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    # Применение знака градиента к изображению
    perturbed_image = image + epsilon * np.sign(gradient)
    perturbed_image = np.clip(perturbed_image, 0, 1)  # Убедиться, что значения остаются в пределах [0, 1]
    return perturbed_image

# Генерация противоречивых примеров для модели
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []

    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape(1, 28, 28, 1), dtype=tf.float32)  # Преобразование в tensor
        label = tf.convert_to_tensor(labels[i].reshape(1, 10), dtype=tf.float32)  # Преобразование метки в tensor

        # Вычисление градиента
        with tf.GradientTape() as tape:
            tape.watch(image)  # Отслеживание изменений изображения
            prediction = model(image)  # Прогноз от модели
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)  # Потери
        gradient = tape.gradient(loss, image)  # Градиент потерь относительно изображения

        # Генерация атакующего изображения
        adv_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())  # Преобразуем в numpy
        adversarial_images.append(adv_image.reshape(28, 28))  # Добавляем изображение в список

    return np.array(adversarial_images)

# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)

**3. Оценка противоречивых примеров на обеих моделях**


In [14]:

# Оценка первой модели на противоречивых примерах
test_labels_argmax = np.argmax(test_labels, axis=1) # Преобразование onehot меток в целые числа
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')
# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1,
28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped,
test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0787 - loss: 7.0997
Accuracy of model1 on adversarial examples: 0.10090000182390213
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9488 - loss: 0.1605
Accuracy of model2 on adversarial examples from model1: 0.9567999839782715


**4. Оценка модели на противоречивых примерах**


In [15]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)
# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28,
28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8504 - loss: 0.4549
Accuracy of model1 on adversarial examples from model2: 0.8680999875068665


**Вывод**



Модель model1 продемонстрировала точность 85.04% на стандартных (не атакующих) изображениях из набора данных MNIST.
Когда модель была протестирована на противоречивых примерах, созданных с использованием FGSM атаки, точность составила 86.81%.
Этот результат предполагает, что модель на этих атакующих примерах показала чуть большую точность, чем на стандартных, что может указывать на то, что атака не вносит значительных изменений в изображения при текущем параметре epsilon или модель хорошо справляется с такими небольшими искажениями.